**Question One**

In [45]:
#Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math
from shapely.geometry import Point
import geopy.distance
import pyproj
import os
from math import radians, cos, sin, asin, sqrt

In [46]:
# Import data

df = pd.read_csv('/Users/sshul/Documents/GitHub/geospatial-data-science/labs/lab5/lab5/seattle_house_prices.csv')

houses = len(df)
print(f'There are {houses} houses in this dataset')

There are 19451 houses in this dataset


In [47]:
df.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'yr_built',
       'lat', 'long'],
      dtype='object')

In [48]:
#Convert DataFreame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']))
gdf.crs = "EPSG:4326"
gdf_utm = gdf.to_crs('EPSG:32610')

In [49]:
gdf_utm.info()
print(f'There are six features for predicting price: number of bedrooms, number \n of bathrooms, square feet of livingspace, total square feet, year built, \n and location.')
print('There are not any null values.')

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19451 entries, 0 to 19450
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   price        19451 non-null  int64   
 1   bedrooms     19451 non-null  int64   
 2   bathrooms    19451 non-null  float64 
 3   sqft_living  19451 non-null  int64   
 4   sqft_lot     19451 non-null  int64   
 5   yr_built     19451 non-null  int64   
 6   lat          19451 non-null  float64 
 7   long         19451 non-null  float64 
 8   geometry     19451 non-null  geometry
dtypes: float64(3), geometry(1), int64(5)
memory usage: 1.3 MB
There are six features for predicting price: number of bedrooms, number 
 of bathrooms, square feet of livingspace, total square feet, year built, 
 and location.
There are not any null values.


In [50]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix['price'].sort_values(ascending = False)

price          1.000000
sqft_living    0.702296
bathrooms      0.524395
bedrooms       0.315804
lat            0.308082
sqft_lot       0.090125
yr_built       0.052453
long           0.020092
Name: price, dtype: float64

In [51]:
answer = f'''The three best correlated values with the housing price 
are square feet of livingspace (.702), number of bathrooms (.524), and 
number of bedrooms (.315). The three worst correlated variables are 
square feet of the lot (.09), year built (.05), and location (.308 lat, 
.02 lon).'''
print(answer)

The three best correlated values with the housing price 
are square feet of livingspace (.702), number of bathrooms (.524), and 
number of bedrooms (.315). The three worst correlated variables are 
square feet of the lot (.09), year built (.05), and location (.308 lat, 
.02 lon).


**Question Two** use distance to location potentially

(alterations)

In [52]:
# Prepare datasets
parksfilepath = 'lab5\ToUpload\Seattle_Parks_and_Recreation_Parks_Features.csv' 
dfparks = pd.read_csv(parksfilepath)
dfparks = dfparks[dfparks['xPos'].notna()]
dfparks = dfparks[dfparks['yPos'].notna()]

hospitalsfilepath = 'lab5\ToUpload\Hospitals.csv'
df_hospitals = pd.read_csv(hospitalsfilepath)
wa_hospitals = df_hospitals.loc[df_hospitals['STATE'] == 'WA']
wa_hospitals = wa_hospitals.reset_index(drop=True)

schoolsfilepath = 'lab5\ToUpload\Public_Schools.csv'
df_schools = pd.read_csv(schoolsfilepath)
wa_schools = df_schools.loc[df_schools['STATE'] == 'WA']
wa_schools = wa_schools.reset_index(drop=True)

In [53]:
## All functions

    #distance functions
    
def distance_to_place(pcoords):
    '''
    This function will calculate the distance of each house in gdf_utm to a given place and return a list of 
    each distance in kilometers
    '''
    dist = []
    numdist = []
    
    for loc in gdf_utm['coords']:
        d = geopy.distance.distance(pcoords, loc)
        dist.append(d)
        
    for item in dist:
        numdist.append(float((str(item).split(' ')[0])))
        
    return numdist


def dist(hloc, ploc):
    '''This function will calculate the geodesic distance between the given house and the given place'''
    
    # convert decimal degrees to radians 
    hlat = float(hloc[0])
    hlon = float(hloc[1])
    plat = float(ploc[0])
    plon = float(ploc[1])
    
    hlat, hlon, plat, plon = map(radians, [hlat, hlon, plat, plon])

    # haversine formula 
    dlon = plon - hlon
    dlat = plat - hlat

    a = sin(dlat/2)**2 + cos(hlat) * cos(plat) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))

    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km


def get_pdist(dfr):
    ''' This function will calculate the distance from the house to its closest park'''
    distlist = []
    
    for parkloc in dfparks['coords']:
        d = dist(dfr['coords'], parkloc)
        distlist.append(d)
        
    m = min(distlist)
        
    return m


def get_hdist(dfr):
    ''' This function will calculate the distance from the house to its closest hospital'''
    distlist = []
    
    for hosloc in wa_hospitals['coords']:
        d = dist(dfr['coords'], hosloc)
        distlist.append(d)
        
    m = min(distlist)
        
    return m


def get_sdist(dfr):
    '''This function will calculate the distance from the house to its closest school'''
    distlist = []
    
    for schloc in wa_schools['coords']:
        d = dist(dfr['coords'], schloc)
        distlist.append(d)
        
    m = min(distlist)
        
    return m


    #coordinate functions
    
def coords(df_row):
    '''This function will create a tuple in the format (lat, lon) of the coordinates of each house.'''
    new = (df_row['lat'], df_row['long'])
    return new


def pcoords(dfr):
    '''This function will create a tuple in the format (lat, lon) of the coordinates of each park.'''
    new = (dfr['yPos'], dfr['xPos'])
    return new


def hcoords(dfr):
    '''This function will create a tuple in the format (lat, lon) of the coordinates of each hospital.'''
    new = (dfr['Y'], dfr['X'])
    return new


def scoords(dfr):
    '''This function will create a tuple in the format (lat, lon) of the coordinates of each school.'''
    new = (dfr['Y'], dfr['X'])
    return new

In [54]:
#Add column of lat,lon points as tuples to each dataset for future ease of use

gdf_utm['coords'] = df.apply(coords, axis=1)
dfparks['coords'] = dfparks.apply(pcoords, axis=1)
wa_hospitals['coords'] = wa_hospitals.apply(hcoords, axis=1)
wa_schools['coords'] = wa_schools.apply(scoords, axis=1)

In [55]:
# Use fuction distance_to_place to get the km value of the distance from each house to the space needle, Seattle international airport, and City Hall.

dist_sn = distance_to_place((47.6205, -122.3493))
gdf_utm['dist_sn'] = dist_sn

dist_SEA = distance_to_place((47.438375, -122.294915))
gdf_utm['dist_SEA'] = dist_SEA

dist_ch = distance_to_place((47.603816, -122.329878))
gdf_utm['dist_ch'] = dist_ch

In [56]:
#Get distance to closest park, hospital, and school
# THIS STEP WILL TAKE A WHILE, I PROMISE IT IS STILL WORKING

gdf_utm['min_park'] = gdf_utm.apply(get_pdist, axis=1)
gdf_utm['min_hospital'] = gdf_utm.apply(get_hdist, axis=1) 
gdf_utm['min_school'] = gdf_utm.apply(get_sdist, axis=1)  

(model)

In [67]:
# Define feature list
feature_list =  ['sqft_living', 'bathrooms', 'bedrooms', 
                 'lat', 'sqft_lot', 'yr_built', 'long', 'dist_sn', 'dist_SEA',
                 'dist_ch', 'min_park', 'min_school', 'min_hospital']

# Define features and labels 
X = gdf_utm[feature_list]
y = gdf_utm['price']

# Standarize data
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)

In [68]:
# Split data 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [69]:
# Define model
forest_reg = RandomForestRegressor(n_estimators = 30)

# Fit model
forest_reg.fit(X_train, y_train)

RandomForestRegressor(n_estimators=30)

(model evaluation)

In [70]:
# Predict test labels predictions
predictions = forest_reg.predict(X_test)

# Compute mean-squared-error
final_mse = mean_squared_error(y_test , predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

154296.18752387114

In [71]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix['price'].sort_values(ascending = False)

price           1.000000
sqft_living     0.702296
bathrooms       0.524395
bedrooms        0.315804
lat             0.308082
dist_SEA        0.170298
sqft_lot        0.090125
min_school      0.064401
yr_built        0.052453
long            0.020092
min_hospital   -0.107419
min_park       -0.187057
dist_ch        -0.288018
dist_sn        -0.291519
Name: price, dtype: float64